In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from pathlib import Path
import nltk
import string
import json
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
"""
The following code reads the inverted_index that was already created and
stored before. The data is stored in the form of bytes by the pickle library.
"""

f = open("inverted_index.txt","rb")
txt = f.read()
f.close()
inverted_index = pickle.loads(txt)

f = open("id_to_name.txt","rb")
txt = f.read()
f.close()
id_to_name = pickle.loads(txt)

f = open("name_to_id.txt","rb")
txt = f.read()
f.close()
name_to_id = pickle.loads(txt)

#......

In [ ]:
# creating universal_docs variable as a support to NOT queries
global universal_docs
universal_docs = [i for i in range(1,1134)]

In [ ]:
"""
The function [process_query] processes the query as follows:
1. lowercase the query
2. remove punctuation
3. tokenize the query
4. stopword removal (english stopword removal)
5. remove blankspace tokens

Arguments : Query (String)
Returns : Query Tokens (List)
"""

def process_query(query):
    query = query.lower()
    query = query.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(query)
    tokens_without_sw = [word for word in tokens if not word in stopwords.words('english')]
    return tokens_without_sw

#.....

In [ ]:
"""
The function [process_NOT] provides support for the 'NOT' operation in the input
operation sequence. In the retrieval pipeline of many functions, this function is
called first to process the 'NOT' operation. The 'OR' and 'AND' operations are
handled after this function.

Arguments : Query Tokens is the list of tokens (List)
            Query Sequence is the input operation sequence (List)
            Inverted Index is the unigram inverted index (Dictionary)
Returns : tuple with 2 elements
          Processed_Postings is the list of postings from the query tokens after 
          the NOT operation (List)
          Processed_Sequence is the operation sequence but with 'NOT' omitted
          because NOT has been processed (List)
"""

def process_NOT(query_tokens, query_sequence, inverted_index):
    processed_query_postings = []
    processed_query_sequence = query_sequence.copy()
    universal_local = set(universal_docs)
    n = len(query_tokens)
    m = len(processed_query_sequence)
    first = False
    if (n==m):
        if (processed_query_sequence[0]!='NOT'):
            print("Cant Process Query")
            return []
        else :
            try:
                current_set = set(inverted_index[query_tokens[0]])
            except KeyError:
                processed_query_postings.append([])
            else:
                rem_list = list(universal_local - current_set)
                processed_query_postings.append(rem_list)
            processed_query_sequence.pop(0)
            first = True

    if (first == False):
        current_list = inverted_index[query_tokens[0]]
        processed_query_postings.append(current_list)

    m = len(processed_query_sequence)

    for i in range(m):
        operation = processed_query_sequence[i]
        operation_str = operation.split(" ")
        if 'NOT' not in operation_str:
            try:
                current_list = inverted_index[query_tokens[i+1]]
            except KeyError:
                processed_query_postings.append([])  
            else:  
                processed_query_postings.append(current_list)
            continue
        if operation_str[-1] != 'NOT':
            print(operation_str)
            print("Cant Process Query Fail 1")
            return None
        if len(operation_str) != 2:
            print("Cant Process Query Fail 2")
            return None
        try:
            current_set = set(inverted_index[query_tokens[i+1]])
        except:
            processed_query_postings.append([])
        else:    
            rem_list = list(universal_local-current_set)
            processed_query_postings.append(rem_list)
        new_operation = operation_str[0]
        processed_query_sequence[i]=new_operation

    return (processed_query_postings,processed_query_sequence)

#.....

In [ ]:
#Binary Search Function to minimise the comparisons while retrieving
def binary_search_count(tofind,inlist):
    count = 0
    list_len = len(inlist)
    
    lower = 0
    upper = list_len - 1
    mid = int((lower+upper)/2)
    # print("Finding",tofind)
    while True:
        # print("lower",lower,end =' ')
        # print("middle",mid, end =' ')
        # print("upper",upper)
        count+=1
        if (inlist[mid]==tofind):
            # print("Found")
            return (True,count)
        if (upper <= lower):
            # print("Not Found")
            return (False,count)
        if (inlist[mid]>tofind):
            upper = mid - 1
            mid = int((lower+upper)/2)
        elif (inlist[mid]<tofind):
            lower = mid + 1
            mid = mid = int((lower+upper)/2)
        
#.....        

In [ ]:
"""
The function [generate_postings_OR] combines two postings list in a
union operation. It also counts the number of comparisons done to generate such
union.

Arguments : First Postings List (List)
            Second Postings List (List)

Returns :   tuple with 2 elements
            Union of the two lists (List)
            Count of Comparisons (Integer)
"""

def generate_postings_OR(first_list,second_list):
    count = 0
    f_len = len(first_list)
    s_len = len(second_list)

    first_list.sort()
    second_list.sort()
    if (f_len<=s_len):
        findfrom = first_list
        findin = second_list
    else:
        findfrom = second_list
        findin = first_list
    
    new_list = findin.copy()
    findin_new = findin.copy()

    for i in findfrom:
        # if i not in findin:
        #     new_list.append(i)
        result = binary_search_count(i,findin_new)
        count += result[1]
        if (result[0]==False):
            new_list.append(i)
        if (result[0]==True):
            index = findin_new.index(i)
            findin_new = findin_new[index:]
    new_list.sort()
    
    return (new_list,count)

#.....

In [ ]:
"""
The function [generate_postings_AND] is similar to the function [generate_postings_OR].
It returns intersection of the two lists and the count of comparisons.
"""

def generate_postings_AND(first_list,second_list):
    count = 0
    f_len = len(first_list)
    s_len = len(second_list)
    first_list.sort()
    second_list.sort()
    if (f_len<=s_len):
        findfrom = first_list
        findin = second_list
    else:
        findfrom = second_list
        findin = first_list
    
    new_list = []
    findin_new = findin.copy()
    for i in findfrom:
        result = binary_search_count(i,findin_new)
        count += result[1]
        if (result[0]==True):
            new_list.append(i)
            index = findin_new.index(i)
            findin_new = findin_new[index:]
    
    return (new_list,count)

#.....

In [ ]:
"""
The function [process_OR] provides support for 'OR' operation in the 
input operation sequence. This function is called after [process_NOT].
It calculates union operation for wherever 'OR' is present in the sequence.

Arguments : Query Postings : The list of postings of queries after NOT (List)
            Query Sequence : Sequence consisting of just AND and OR (List)
Returns : tuple of 3 elements
          List of Postings after processing OR (List)
          List of operation sequence containing just AND (List)
          Count of comparisons done to process OR (Integer)
"""

def process_OR(processed_query_postings,query_sequence):
    count = 0
    new_postings = processed_query_postings.copy()
    new_sequence = query_sequence.copy()
    while 'OR' in new_sequence:
        index = new_sequence.index('OR')
        first = new_postings[index]
        second = new_postings[index+1]
        result = generate_postings_OR(first,second)
        new_sequence.pop(index)
        new_postings.insert(index,result[0])
        new_postings.pop(index+1)
        new_postings.pop(index+1)
        count+=result[1]

    return (new_postings,new_sequence,count)      

In [ ]:
"""
The function [process_AND] provides support for 'AND' operation in the input
operation sequence. This function is called after [process_OR] i.e it is the
last operation processing function. 

Arguments : Query Postings : List of postings after processing OR (List)
            Query Sequence : List of operations containing just AND (List)
            Count of comparisons to keep track of comparisons made in AND
            since this is a recursive function (Integer)

Returns : tuple of 3 elements
          Final Postings List of the query and input operation given by user (List)
          Empty Query Sequence since all operations are processed (List)
          Count : Total Count of all comparisons (Integer)

To minimise the number of comparisons, this function first sorts the 
query postings by their frequency and then calls the [generate_postings_AND].
"""
def process_AND(processed_query_postings,query_sequence,count):
    n = len(processed_query_postings)
    m = len(query_sequence)
    dictionary = {}
    if (n != m + 1):
        print("Error in Preprocessing")
        return None

    if len(query_sequence)==0:
        return (processed_query_postings,query_sequence,count)
   
    # new_postings = processed_query_postings.copy()
    new_postings = []
    new_sequence = query_sequence.copy()
    for i in range(n):
        dictionary[i]=len(processed_query_postings[i])
    dictionary = dict(sorted(dictionary.items(), key=lambda item: item[1]))
    # print(dictionary)
    for i in dictionary:
        new_postings.append(processed_query_postings[i])
    first = new_postings[0]
    second = new_postings[1]
    result = generate_postings_AND(first,second)
    new_sequence.pop(0)
    new_postings.insert(0,result[0])
    new_postings.pop(1)
    new_postings.pop(1)
    count+=result[1]

    return process_AND(new_postings,new_sequence,count)

#.....


In [ ]:
"""
The function [boolean_retrieval] performs the retrieval of input query and 
input operation sequence. It uses the functions [process_query], [process_NOT],
[process_OR] and [process_AND]

Arguments : Query (String)
            Input Operation Sequence (List)
            Inverted Index (Dictionary)
Returns : Final List of Documents containing required query (List)
          Count of comparisons done to retrieve the list (Integer)
"""

def boolean_retrieval(query,query_sequence,inverted_index):
    query_tokens = process_query(query)
    result1 = process_NOT(query_tokens, query_sequence, inverted_index)
    processed_postings = result1[0]
    new_sequence = result1[1]

    count = 0
    result2 = process_OR(processed_postings,new_sequence)
    processed_postings = result2[0]
    new_sequence = result2[1]
    count+= result2[2]

    # print("After Processing OR count of comparisons",count)
    # print("len",len(processed_postings[0]))
    result3 = process_AND(processed_postings,new_sequence,0)
    final_result = result3[0][0]
    final_sequence = result3[0]
    count += result3[2]

    # print("After Processing AND count of comparisons",count)

    # print("Number of Documents Retrieved",len(final_result))
    return (final_result,count)
    
#.....

In [1]:
N = int(input("Enter Number of Queries"))
for i in range(N):
    query = input("Enter Query Line 1\n")
    operation_sequence = input("Enter Query Line 2 (Separated by Commas)\n")
    operation_sequence = operation_sequence.upper()
    operation_sequence = operation_sequence.split(",")
    result = boolean_retrieval(query,operation_sequence,inverted_index)
    print("Number of Documents Retrieved",len(result[0]))
    print("Number of Comparisons",result[1])


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=04d84d63-ff1d-4672-9f78-d034a2868658' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>